<a href="https://colab.research.google.com/github/S-cyber-cell/PROJECS/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





In [35]:

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# New section

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [37]:
import zipfile

zip_path = "/content/a.zip"

# Check if ZIP file is valid
try:
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        bad_file = zip_ref.testzip()
        if bad_file:
            print(f"Corrupt file found inside ZIP: {bad_file}")
        else:
            print("ZIP file is valid!")
except zipfile.BadZipFile:
    print("The ZIP file is corrupted. Try re-downloading it.")


ZIP file is valid!


Data Pre-processing

In [41]:
import zipfile
import os

# Path to your downloaded ZIP file
zip_path = "/content/a.zip"  # Change this to your actual filename

# Check if the file exists
if not os.path.exists(zip_path):
    raise FileNotFoundError(f"The file '{zip_path}' does not exist. Please check the path and filename.")

# Extract it
try:
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("dataset")  # Extract to 'dataset' folder
    print("Extraction completed!")
except zipfile.BadZipFile:
    raise zipfile.BadZipFile(f"The file '{zip_path}' is not a valid ZIP file. It may be corrupted or a different file type.")

extract_folder = "extracted_data"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"ZIP file extracted to: {extract_folder}")

# news_dataset = pd.read_csv('/train.csv') #Change the path if the dataset file is inside the dataset folder

# Get list of files in the extracted folder
files = os.listdir(extract_folder)  # Define 'files' here


# Find Excel file
excel_files = [f for f in files if f.endswith(".xlsx") or f.endswith(".xls")]
if excel_files:
    excel_path = os.path.join(extract_folder, excel_files[0])  # Use the first Excel file
    df = pd.read_excel(excel_path)

    # Convert Excel to CSV
    csv_path = excel_path.replace(".xlsx", ".csv").replace(".xls", ".csv")
    df.to_csv(csv_path, index=False)

    print(f"Excel converted to CSV: {csv_path}")


Extraction completed!
ZIP file extracted to: extracted_data
Excel converted to CSV: extracted_data/bharatfakenewskosh (3).csv


In [49]:
import pandas as pd
df = pd.read_excel(r"/content/extracted_data/bharatfakenewskosh (3).xlsx")

# Display first few rows
print(df.head())
# import zipfile
# import pandas as pd

# # Replace with the correct path
# zip_path = "/content/a.zip"

# # Extract the ZIP file
# with zipfile.ZipFile(zip_path, "r") as zip_ref:
#     zip_ref.extractall("extracted_data")

# # Read the CSV file from the extracted data
# csv_path = "/content/extracted_data/bharatfakenewskosh (3).xlsx" # Change this if your CSV has a different name
# df = pd.read_csv(csv_path)

       id         Author_Name Fact_Check_Source Source_Type  \
0  BFNK_1  Shinjinee Majumder          Alt News        IFCN   
1  BFNK_2         Kalim Ahmed          Alt News        IFCN   
2  BFNK_3      Abhishek Kumar          Alt News        IFCN   
3  BFNK_4      Abhishek Kumar          Alt News        IFCN   
4  BFNK_5              Kinjal          Alt News        IFCN   

                                           Statement  \
0  फ़ैक्ट-चेक: तेलंगाना में एक रिपोर्टर ने गृह मंत...   
1  PM मोदी को UAE का सर्वोच्च नागरिक सम्मान मिलने...   
2  वायरल तस्वीर में सुप्रीम कोर्ट के जज सूर्यकांत...   
3  मीडिया ने दी ग़लत ख़बर: कटनी में मुस्लिम सरपंच क...   
4  महिला ने राहुल गांधी को कश्मीर मुद्दे पर मोदी ...   

                                 Eng_Trans_Statement  \
0  Fact-check: A reporter in Telangana stopped sp...   
1  Share by stating the old video of PM Modi's hi...   
2  Supreme Court judges Suryakant and JB Pardiwal...   
3  Media gave wrong news: After the victory of Mu...   
4  T

In [50]:
df.shape

(26232, 19)

In [51]:
# counting the number of missing values in the dataset
df.isnull().sum()

,0
id,0
Author_Name,227
Fact_Check_Source,0
Source_Type,0
Statement,0
Eng_Trans_Statement,0
News Body,873
Eng_Trans_News_Body,872
Media_Link,1743
Publish_Date,815


In [52]:
# replacing the null values with empty string
df = df.fillna('')

In [53]:
# merging the author name and news title
df['content'] = df['Author_Name']+' '+df['Text']

In [54]:

print(df['News Body'])

0        सोशल मीडिया पर एक वीडियो वायरल है जिसमें एक पत...
1        प्रधानमंत्री नरेंद्र मोदी को सोने की चेन से सम...
2        बीते दिनों नूपुर शर्मा ने टीवी डिबेट में पैगम्...
3        एक वीडियो सोशल मीडिया पर वायरल है. इसे शेयर कर...
4        सोशल मीडिया पर राहुल गांधी का एक वीडियो वायरल ...
                               ...                        
26227    The situation in Ukraine is getting tougher. M...
26228    After the formation of the Aam Aadmi Party gov...
26229    Societymedia is a web of misleading and fake i...
26230    Amidst the invasion of Ukraine, a big claim ha...
26231    A picture is becoming massively viral on Socie...
Name: News Body, Length: 26232, dtype: object


In [55]:

# separating the data & label
X = df.drop(columns='Label', axis=1)
Y = df['Label'] #-->the output

In [56]:
print(X)
print(Y)

               id             Author_Name Fact_Check_Source Source_Type  \
0          BFNK_1      Shinjinee Majumder          Alt News        IFCN   
1          BFNK_2             Kalim Ahmed          Alt News        IFCN   
2          BFNK_3          Abhishek Kumar          Alt News        IFCN   
3          BFNK_4          Abhishek Kumar          Alt News        IFCN   
4          BFNK_5                  Kinjal          Alt News        IFCN   
...           ...                     ...               ...         ...   
26227  BFNK_26228          Parul Tewari -         Scroll.in    Non_IFCN   
26228  BFNK_26229  Nisar Ahmed Siddiqui -         Scroll.in    Non_IFCN   
26229  BFNK_26230          Parul Tewari -         Scroll.in    Non_IFCN   
26230  BFNK_26231          Dilshad Noor -         Scroll.in    Non_IFCN   
26231  BFNK_26232          Dilshad Noor -         Scroll.in    Non_IFCN   

                                               Statement  \
0      फ़ैक्ट-चेक: तेलंगाना में एक रिपोर

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [57]:
# def stemming(content):
#     stemmed_content = re.sub('[^a-zA-Z]',' ',content)         #exceprt this all will be replaced by " "
#     stemmed_content = stemmed_content.lower()
#     stemmed_content = stemmed_content.split() #to split the para to words w/o tokenization
#     stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#     stemmed_content = ' '.join(stemmed_content)
#     return stemmed_content
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('punkt_tab') # Download the 'punkt_tab' dataset

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(Text):
    # Remove URLs
    Text = re.sub(r"http\S+|www\S+|https\S+", "", Text, flags=re.MULTILINE)

    # Remove HTML tags
    Text = re.sub(r"<.*?>", "", Text)

    # Remove special characters, numbers, and extra spaces
    Text = re.sub(r"[^a-zA-Z]", " ", Text)

    # Convert to lowercase
    Text = Text.lower()

    # Tokenize
    words = word_tokenize(Text)

    # Remove stopwords & short words, apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 2]

    return " ".join(words)

# # Apply the function
# df["clean_text"] = df["Text"].apply(clean_text)
# print(df.head())



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [77]:
!pip install tpot

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.8

In [84]:
!pip install --upgrade numpy scipy scikit-learn tpot


  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.


In [89]:
# Install necessary packages
# !pip install --upgrade numpy pandas transformers torch sklearn nltk imbalanced-learn

import pandas as pd
import numpy as np
import torch
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from imblearn.over_sampling import SMOTE

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Load dataset
df["clean_text"] = df["Text"].apply(clean_text)

if 'clean_text' not in df.columns:
    raise ValueError("Column 'clean_text' not found in dataset.")

X_train, X_test, y_train, y_test = train_test_split(df["clean_text"], df["Label"], test_size=0.2, random_state=42)

# Apply SMOTE for balancing the dataset
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = FakeNewsDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = FakeNewsDataset(X_test.tolist(), y_test.tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# Load DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = Adam(model.parameters(), lr=2e-5)

# Training function
def train_model(model, train_loader, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

# Train model
train_model(model, train_loader, optimizer)

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return accuracy_score(true_labels, predictions)

# Evaluate the model
accuracy = evaluate_model(model, test_loader)
print(f"BERT Model Accuracy: {accuracy:.2f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.6712
Epoch 2, Loss: 0.6700
Epoch 3, Loss: 0.6697
BERT Model Accuracy: 0.59


In [63]:
#separating the data and label
X = df['Text'].values
Y = df['Label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [64]:
print(Y)

[False False False ...  True False  True]


In [65]:
Y.shape

(26232,)

In [66]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [67]:
print(X)

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 0)	1.0
  (12, 0)	1.0
  (13, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  :	:
  (26207, 0)	1.0
  (26208, 0)	1.0
  (26209, 1)	1.0
  (26210, 0)	1.0
  (26211, 0)	1.0
  (26212, 0)	1.0
  (26213, 0)	1.0
  (26214, 0)	1.0
  (26215, 0)	1.0
  (26216, 0)	1.0
  (26217, 0)	1.0
  (26218, 0)	1.0
  (26219, 1)	1.0
  (26220, 0)	1.0
  (26221, 1)	1.0
  (26222, 0)	1.0
  (26223, 0)	1.0
  (26224, 0)	1.0
  (26225, 0)	1.0
  (26226, 0)	1.0
  (26227, 0)	1.0
  (26228, 0)	1.0
  (26229, 0)	1.0
  (26230, 0)	1.0
  (26231, 0)	1.0


Splitting the dataset to training & test data

In [68]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [69]:
model = LogisticRegression()

In [70]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [71]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [72]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.6066237788896831


In [73]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [74]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.6066323613493425


Making a Predictive System

In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [ ]:
print(Y_test[3])

0


In [90]:
model.save_pretrained("fake_news_bert_model")  # Save to a directory named "fake_news_bert_model"


In [91]:
import joblib

# ... (your training code) ...

# Save the model's state dictionary using Joblib
joblib.dump(model.state_dict(), "fake_news_bert_model.joblib")

['fake_news_bert_model.joblib']

In [92]:
import joblib
from transformers import DistilBertForSequenceClassification

# Create a new instance of the model
loaded_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2) # Assuming 2 labels

# Load the saved state dictionary
state_dict = joblib.load("fake_news_bert_model.joblib")

# Load the state dictionary into the model
loaded_model.load_state_dict(state_dict)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>